In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator
import glob
import xarray as xr

import ipywidgets as widgets

import sys
import os
sys.path.insert(1, '../Modules/')
from HW_detection_algorithm import *

In [2]:
# 1 SET-UP PARAMETERS: Choose how to define the heatwaves #

In [3]:
# Pick a threshold for heatwaves (e.g. 90th percentile) #

threshold_widget=widgets.Dropdown(
    options=[85, 90, 95, 99],
    value=90,
    description='Threshold:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

threshold_widget

Dropdown(description='Threshold:', index=1, layout=Layout(width='200px'), options=(85, 90, 95, 99), style=Desc…

In [4]:
# Pick a minimum duration (e.g. 3 days) #

duration_widget = widgets.IntSlider(
    value=3,                    # Default value
    min=3,                      # Minimum duration
    max=20,                     # Maximum duration  
    step=1,                     # Step size
    description='Min Duration:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

duration_widget

IntSlider(value=3, description='Min Duration:', layout=Layout(width='400px'), max=20, min=3, style=SliderStyle…

In [5]:
# These ones are fixed for now ;) #
var="tmax" #tmax,tmin

percent_thresh=threshold_widget.value
duration_min=duration_widget.value
print (percent_thresh)
### TIME ###
clim_start=1993 #1981
clim_end=2016 #2010
period_start=1993
period_end=2022
if (period_end<clim_end) or (period_start>clim_start):
    raise Exception("Climatology period falls outside of target period")
years=[*range(period_start,period_end+1)]
climyears=[*range(clim_start,clim_end+1)]

nyear=len(years)
nclimyear=len(climyears)
nday=123 # days in period May to August
nrealisation=1 # ensemble members

90


In [6]:
# 2 LOCATION: Choose by longitude and latitude #
# What si the record of heatwaves in Cluj-Napoca? How about where you want to go on holiday this year? 
# Europe only...global model under construction! 

In [7]:
# Pick a city...or add your own coordinates 

#region="Milano"
#lon='''09°11'24"E'''
#lat='''45°28'01"N'''

#region="London"
#lon = '''0°7'39"W'''
#lat = '''51°30'26"N'''

#region="Paris"
#lon = '''2°21'8"W'''
#lat = '''48°51'24"N'''

# Cluj-Napoca #
region="Cluj-Napoca"
lon='''23°33'00"E'''
lat='''46°46'00"N'''

deg, minutes, seconds, direction =  re.split('[°\'"]', lat)
latP= (float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['W', 'S'] else 1)
deg, minutes, seconds, direction =  re.split('[°\'"]',lon)
lonP= (float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['W', 'S'] else 1)

# Find nearest ERA5 cell #
def find_nearest(array, value):
    #array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]


In [8]:
# 3 DATA: Open/extract ERA5 temperature data #

In [11]:
# Open ERA5 temperature data #

files=sorted(glob("../DATA/ERA5_tmax_*_EUR_MJJA.nc"))
ds = xr.open_mfdataset(files, combine='by_coords')
print (files)
lons=ds.lon.values
lats=ds.lat.values
ind_lon=np.argwhere(lons==find_nearest(lons,lonP))[0]
ind_lat=np.argwhere(lats==find_nearest(lats,latP))[0]
t_data=ds[var].values[:,ind_lat,ind_lon].reshape(nyear,nday,1)


['../DATA/ERA5_tmax_1993_EUR_MJJA.nc', '../DATA/ERA5_tmax_1994_EUR_MJJA.nc', '../DATA/ERA5_tmax_1995_EUR_MJJA.nc', '../DATA/ERA5_tmax_1996_EUR_MJJA.nc', '../DATA/ERA5_tmax_1997_EUR_MJJA.nc', '../DATA/ERA5_tmax_1998_EUR_MJJA.nc', '../DATA/ERA5_tmax_1999_EUR_MJJA.nc', '../DATA/ERA5_tmax_2000_EUR_MJJA.nc', '../DATA/ERA5_tmax_2001_EUR_MJJA.nc', '../DATA/ERA5_tmax_2002_EUR_MJJA.nc', '../DATA/ERA5_tmax_2003_EUR_MJJA.nc', '../DATA/ERA5_tmax_2004_EUR_MJJA.nc', '../DATA/ERA5_tmax_2005_EUR_MJJA.nc', '../DATA/ERA5_tmax_2006_EUR_MJJA.nc', '../DATA/ERA5_tmax_2007_EUR_MJJA.nc', '../DATA/ERA5_tmax_2008_EUR_MJJA.nc', '../DATA/ERA5_tmax_2009_EUR_MJJA.nc', '../DATA/ERA5_tmax_2010_EUR_MJJA.nc', '../DATA/ERA5_tmax_2011_EUR_MJJA.nc', '../DATA/ERA5_tmax_2012_EUR_MJJA.nc', '../DATA/ERA5_tmax_2013_EUR_MJJA.nc', '../DATA/ERA5_tmax_2014_EUR_MJJA.nc', '../DATA/ERA5_tmax_2015_EUR_MJJA.nc', '../DATA/ERA5_tmax_2016_EUR_MJJA.nc', '../DATA/ERA5_tmax_2017_EUR_MJJA.nc', '../DATA/ERA5_tmax_2018_EUR_MJJA.nc', '../DATA/ER

In [12]:
# 4 HW DETECTION: Run HW detection algorithm #

In [13]:
# Just click run! #
HW_output=calc_HWMIyear(t_data, climyears, years, cross_valid = False, percent_thresh = percent_thresh, duration_min = duration_min, opt="polyfit")

In [14]:
# 5 SAVE: Save data for plotting in notebook 2 #

In [15]:
# Outputs number of HW days from May to July #
hw_days = np.sum(HW_output['HW_occurrence'], axis=1)
df = pd.DataFrame({
    'year': years,
    'NumberHWDays': hw_days
})
filename = f"Output/NumberHWdays_ERA5_{region}_thresh{percent_thresh}_dur{duration_min}.csv"
if os.path.exists(filename):
    os.remove(filename)
df.to_csv(filename, index=False)
print(f"Saved heat wave days data to {filename}")

Saved heat wave days data to Output/NumberHWdays_ERA5_Cluj-Napoca_thresh90_dur3.csv


In [15]:
# 6 paleo-simulation "past2k" DATA  - needed for the feature selection later

ds2 = xr.open_dataset("../DATA/past2k_tasmax_HWs_EUR_MJJA_period70018850_clim88218850.nc", decode_times=False)

past2k_HWs=ds2['NDQ90_May'].values+ds2['NDQ90_Jun'].values+ds2['NDQ90_Jul'].values
print (past2k_HWs.shape)
lons=ds2.lon.values
lats=ds2.lat.values
ind_lon=int(np.argwhere(lons==find_nearest(lons,lonP))[0])
ind_lat=int(np.argwhere(lats==find_nearest(lats,latP))[0])

past2k_HWs_target=past2k_HWs[:,0,ind_lat,ind_lon]

dataset_out=xr.DataArray(past2k_HWs_target, dims=["year"], name="NumberHWDays")
dataset_out.to_netcdf(f"NumberHWdays_past2k_{region}.nc")

df = pd.DataFrame({
    'year': range(0,1850,1),
    'NumberHWDays': past2k_HWs_target
})

# Create filename
filename = f"Output/NumberHWdays_past2k_{region}_thresh90_dur3.csv"

# Remove existing file if it exists
if os.path.exists(filename):
    os.remove(filename)

# Save to CSV
df.to_csv(filename, index=False)

print(f"Saved heat wave days data to {filename}")


(1850, 1, 26, 41)
Saved heat wave days data to Output/NumberHWdays_past2k_Cluj-Napoca_thresh90_dur3.csv


/tmp/ipykernel_3507210/3025596213.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ind_lon=int(np.argwhere(lons==find_nearest(lons,lonP))[0])
/tmp/ipykernel_3507210/3025596213.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ind_lat=int(np.argwhere(lats==find_nearest(lats,latP))[0])
